In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error,mean_absolute_error
import datetime as dt
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import preparing_data as F

In [2]:
data=pd.read_pickle("./dataframe/df_20210912_003323_ONLY_RISKY_EVENTS-6.pkl")
data.reset_index(inplace=True)
data.drop(['index'], inplace=True, axis=1)
print(data.shape)
data.head()

(322, 77)


,event_id,MISS_DISTANCE,RELATIVE_SPEED,RELATIVE_POSITION_R,RELATIVE_POSITION_T,RELATIVE_POSITION_N,RELATIVE_VELOCITY_R,RELATIVE_VELOCITY_T,RELATIVE_VELOCITY_N,COLLISSION_PROBABILITY,...,OBJECT2_CORR_CRDOT_N,OBJECT2_CORR_CTDOT_R,OBJECT2_CORR_CTDOT_T,OBJECT2_CORR_CTDOT_N,OBJECT2_CORR_CTDOT_RDOT,OBJECT2_CORR_CNDOT_R,OBJECT2_CORR_CNDOT_T,OBJECT2_CORR_CNDOT_N,OBJECT2_CORR_CNDOT_RDOT,OBJECT2_CORR_CNDOT_TDOT
0,37,2244.0,11172.0,87.9,-1474.9,1689.9,-201.4,-8424.3,-7336.0,-3.418392,...,0.002843,-0.482607,0.258107,0.017817,-0.254852,0.237226,-0.327443,0.466642,0.327803,0.238005
1,3555,237.0,14919.0,-233.2,7.5,-41.6,-12.0,-14836.7,-1568.3,-3.551758,...,0.566286,-0.999994,-0.963072,0.449493,0.954689,-0.580151,-0.720319,0.958021,0.719180,0.581045
2,5773,966.0,212.0,191.0,-944.8,-68.9,33.8,-8.4,209.3,-3.711974,...,-0.519912,-0.999938,0.263666,0.533750,-0.419885,-0.203647,0.477408,0.871159,-0.506729,0.210922
3,5773,1102.0,212.0,193.3,-1083.1,-75.0,33.9,-8.4,209.3,-3.913996,...,-0.530570,-0.999935,0.266897,0.533748,-0.426298,-0.203569,0.488982,0.871149,-0.517215,0.210876
4,6059,456.0,11707.0,-22.1,288.8,353.3,-10.2,-9077.7,7392.6,-2.055073,...,-0.099861,-0.972160,0.385829,0.382001,-0.372174,0.545381,-0.065783,-0.499656,0.048450,-0.609553


In [3]:
#data=data.head(100)

In [ ]:
data.shape

In [ ]:
############################### FEATURE ENGINEERING ##########################################
# Gradient: Miss distance two last CDM
data["_GRADIENT_MISS_DISTANCE_34"]=(-data.MISS_DISTANCE_3+data.MISS_DISTANCE_4)/abs(data.__time_to_tca_4-data.__time_to_tca_3)
# Gradient: Miss distance first and last CDM
data["_GRADIENT_MISS_DISTANCE_14"]=(-data.MISS_DISTANCE+data.MISS_DISTANCE_4)/abs(data.__time_to_tca_4-data.__time_to_tca)
#Gradient: COLLISSION PROBABILITY two last CDM
data["_GRADIENT_PC_34"]=(-data.COLLISSION_PROBABILITY_3+data.COLLISSION_PROBABILITY_4)/abs(data.__time_to_tca_4-data.__time_to_tca_3)
#Gradient: COLLISSION PROBABILITY first and last CDM
data["_GRADIENT_PC_14"]=(-data.COLLISSION_PROBABILITY+data.COLLISSION_PROBABILITY_4)/abs(data.__time_to_tca_4-data.__time_to_tca)

In [ ]:
data.shape

In [ ]:
#print(list(data.columns))

In [ ]:
aux=data
data=data.loc[:, data.columns.str.endswith('4','TARGET')]
data["COLLISSION_PROBABILITY_TARGET"]=aux.COLLISSION_PROBABILITY_TARGET
print(list(data.columns))
data.shape

In [4]:
train, test = train_test_split(data, test_size=0.30, random_state=42)

In [5]:
print("Train dataframe dimension {} x {}".format(train.shape[0],train.shape[1]))
print("Test dataframe dimension {} x {}".format(test.shape[0],test.shape[1]))

Train dataframe dimension 225 x 77
Test dataframe dimension 97 x 77


In [ ]:
Y_train = train["COLLISSION_PROBABILITY_TARGET"]
X_train= train.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)
Y_test = test["COLLISSION_PROBABILITY_TARGET"]
X_test= test.drop(["COLLISSION_PROBABILITY_TARGET"], axis=1)


In [6]:
Y_train = train["COLLISSION_PROBABILITY"]
X_train= train.drop(["COLLISSION_PROBABILITY"], axis=1)
Y_test = test["COLLISSION_PROBABILITY"]
X_test= test.drop(["COLLISSION_PROBABILITY"], axis=1)


In [7]:
X = X_train
y = Y_train

In [8]:
def bayesian_opt_lgbm(X, y, init_iter=3, n_iters=7, random_state=11, seed = 101, num_iterations = 100):
      dtrain = lgb.Dataset(data=X, label=y)
      #Metric evaluation functions
      def lgb_r2_score(preds, dtrain):                #R2
            labels = dtrain.get_label()
            return 'metric', r2_score(labels, preds), True
      def lgb_root_squared_error(preds, dtrain):      #RMSE
            labels = dtrain.get_label()
            return 'metric', mean_squared_error(labels, preds,squared=False), True
      def lgb_mean_absolute_error(preds, dtrain):     #MAE
            labels = dtrain.get_label()
            return 'metric', mean_absolute_error(labels, preds), True
      def lgb_adjusted_r2_score(preds, dtrain):       #ADJUSTED R2
            labels = dtrain.get_label()
            n=dtrain.num_data()
            k=dtrain.num_feature()
            return 'metric', ((1-r2_score(labels, preds))*(n-1))/(n-k-1), True
            
      # Select metric
      metric='lgb_r2_score'
      metric_feval=lgb_r2_score

      # Objective Function
      def hyp_lgbm(num_leaves, feature_fraction, learning_rate, bagging_fraction, max_depth, min_split_gain, min_child_weight):
              params = {      'application':'regression',
                              'num_iterations': num_iterations,
                              'early_stopping_round':50,
                              'verbose':-1,
                              'metric':metric} # Default parameters
              params["num_leaves"] = int(round(num_leaves))
              params["learning_rate"] = learning_rate
              params['feature_fraction'] = max(min(feature_fraction, 1), 0)
              params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
              params['max_depth'] = int(round(max_depth))
              params['min_split_gain'] = min_split_gain
              params['min_child_weight'] = min_child_weight
              cv_results = lgb.cv(params, dtrain, nfold=5, seed=seed,categorical_feature=[], stratified=False,
                                  verbose_eval =None, feval=metric_feval)
              #print(cv_results)
              return np.max(cv_results['metric-mean'])
    
              # Domain space-- Range of hyperparameters 
      pds = {     'num_leaves': (60, 120),
                  'feature_fraction': (0.1, 0.9),
                  'bagging_fraction': (0.7, 1),
                  'max_depth': (7, 15),
                  'learning_rate':(0.001,0.05), 
                  'min_split_gain': (0.001, 0.1),
                  'min_child_weight': (10, 25)
                  }
      # Surrogate model
      optimizer = BayesianOptimization(hyp_lgbm, pds, random_state=random_state)
                                          
      # Optimize
      optimizer.maximize(init_points=init_iter, n_iter=n_iters)

      filename="./opt_parameters_bo/opt_parameters_df_singles_{}_corr_{}_RISKY_EVENTS-6.pkl".format(dt.datetime.now().strftime("%Y%m%d_%H%M%S"),metric)
      a_file = open(filename, "wb")
      pickle.dump(optimizer.max['params'], a_file)
      a_file.close()

      return optimizer

bayesian_ouput=bayesian_opt_lgbm(X, y, init_iter=5, n_iters=500, random_state=77, seed = 101,num_iterations=300)

|   iter    |  target   | baggin... | featur... | learni... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.3373   |  0.9757   |  0.6138   |  0.03793  |  8.115    |  11.31    |  0.07901  |  79.57    |
|  2        |  0.3779   |  0.8623   |  0.2922   |  0.02773  |  10.2     |  20.73    |  0.08383  |  95.31    |
|  3        |  0.3753   |  0.7888   |  0.3248   |  0.03557  |  10.38    |  10.86    |  0.07496  |  87.14    |
|  4        |  0.3298   |  0.7527   |  0.1395   |  0.01533  |  7.534    |  21.27    |  0.007313 |  85.91    |
|  5        |  0.3679   |  0.8093   |  0.2216   |  0.02779  |  10.55    |  10.54    |  0.08247  |  76.4     |
|  6        |  0.1277   |  0.7515   |  0.2862   |  0.001457 |  14.72    |  11.76    |  0.04174  |  119.2    |
|  7        |  0.3232   |  1.0      |  0.9      |  0.05     |  15.0     |  25.0     |  0.1      |  60.0     |
|  8      

In [9]:
opt_parameters=bayesian_ouput.max['params']
opt_parameters

{'bagging_fraction': 0.9544641620733417,
 'feature_fraction': 0.16571857768977108,
 'learning_rate': 0.04024267703719494,
 'max_depth': 11.505980695478904,
 'min_child_weight': 20.111978836599803,
 'min_split_gain': 0.014320776171270293,
 'num_leaves': 73.39158697018834}

In [10]:
# filename="opt_parameters_balanced_df_adjusted_{}_corr_mae_RISKY_EVENTS-8.pkl".format(dt.datetime.now().strftime("%Y%m%d_%H%M%S"))
# a_file = open(filename, "wb")

# pickle.dump(opt_parameters, a_file)

# a_file.close()

In [11]:
# filename="./opt_parameters_bo/opt_parameters_balanced_df_adjusted_20210911_215733_corr_lgb_r2_score_RISKY_EVENTS-8-lastcdm-BEST-MODEL.pkl"
# a_file = open(filename,"rb")
# output = pickle.load(a_file)
# opt_parameters=output
# output

In [12]:
#optimizer.max['params']

In [13]:
#'bagging_fraction': 1.0, 'feature_fraction': 0.9, 'max_depth': 8.0, 'min_child_weight': 25.0, 'min_split_gain': 0.013771321931506838, 'num_leaves': 88.93816438820497}

In [14]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'regression_l2',
    'learning_rate': opt_parameters.get("learning_rate"),
    'feature_fraction': opt_parameters.get("feature_fraction"),
    'bagging_fraction': opt_parameters.get("bagging_fraction"),
    #'bagging_freq': 10,
    'verbose': -1,
    "max_depth": int(round(opt_parameters.get("max_depth"))),
    "num_leaves": int(round(opt_parameters.get("num_leaves"))),  
    #"max_bin": 512,
    'min_split_gain' : opt_parameters.get("min_split_gain"),
    "num_iterations": 300,
    "n_estimators": 500,
    'min_child_weight' : opt_parameters.get("min_child_weight")
}

In [15]:
#Construct a gradient boosting model.
#gbm = lgb.LGBMRegressor(**hyper_params)

In [16]:
lgbm_train = lgb.Dataset(X, label=y)
#lgbm_eval = lgb.Dataset(X_test, label=Y_test,reference=lgbm_train)

In [17]:
gbm = lgb.train(params=hyper_params,
                train_set=lgbm_train,
                #valid_sets=lgbm_eval,
                #verbose_eval=20,
                #eval_metric='lgb_r2_score',
                #early_stopping_rounds=100
                )

In [18]:
#Build a gradient boosting model from the training set (X, y)
""" gbm.fit(X, y,
        eval_set=[(X_test, Y_test)],
        eval_metric='l1',
        early_stopping_rounds=50) """


" gbm.fit(X, y,\n        eval_set=[(X_test, Y_test)],\n        eval_metric='l1',\n        early_stopping_rounds=50) "

In [19]:
Y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [20]:
# print('The r2 of prediction is:', r2_score(y, Y_pred))
# print('The MSE of prediction is:', mean_squared_error(y, Y_pred, squared=True))
# print('The RMSE of prediction is:', mean_squared_error(y, Y_pred, squared=False))

In [21]:
print('The r2 of prediction is:', r2_score(Y_test, Y_pred))
print('The MSE of prediction is:', mean_squared_error(Y_test, Y_pred, squared=True))
print('The RMSE of prediction is:', mean_squared_error(Y_test, Y_pred, squared=False))
print('The MAE of prediction is:', mean_absolute_error(Y_test, Y_pred))

The r2 of prediction is: 0.7877529347956094
The MSE of prediction is: 0.11301396513034383
The RMSE of prediction is: 0.33617549751631787
The MAE of prediction is: 0.2424334811502857


In [22]:
# df_train = lgb.Dataset(data=X_test, label=Y_test)
# def lgb_adjusted_r2_score(preds, dtrain):
#     labels = dtrain.get_label()
#     n=dtrain.num_data()
#     k=dtrain.num_feature()
#     return 'metric', ((1-r2_score(labels, preds))*(n-1))/(n-k-1), True

In [23]:
#lgb_adjusted_r2_score(Y_pred, df_train)

### Validating model

In [24]:
aux_y=pd.DataFrame(Y_test)
aux_y.reset_index(inplace=True)
aux_y.drop(['index'], inplace=True, axis=1)
aux_y_pred=pd.DataFrame(Y_pred)
aux_y_pred.reset_index(inplace=True)
aux_y_pred.drop(['index'], inplace=True, axis=1)
frames=[aux_y,aux_y_pred]
result=pd.concat(frames,axis=1)
result.columns=["y_true","y_predicted"]
result["y_true_10"]=10**result.y_true
result["y_predicted_10"]=10**result.y_predicted
result

,y_true,y_predicted,y_true_10,y_predicted_10
0,-5.379760,-5.443826,0.000004,0.000004
1,-5.514279,-5.032100,0.000003,0.000009
2,-5.265760,-5.307800,0.000005,0.000005
3,-3.889410,-4.013056,0.000129,0.000097
4,-4.584359,-5.049841,0.000026,0.000009
...,...,...,...,...
92,-5.802995,-5.806139,0.000002,0.000002
93,-5.383000,-5.631494,0.000004,0.000002
94,-4.022231,-4.226447,0.000095,0.000059
95,-5.993962,-5.601549,0.000001,0.000003


In [27]:
result[result["y_true_10"]>0.00001]

,y_true,y_predicted,y_true_10,y_predicted_10
3,-3.889410,-4.013056,0.000129,0.000097
4,-4.584359,-5.049841,0.000026,0.000009
11,-4.974694,-4.997476,0.000011,0.000010
13,-4.767766,-4.738139,0.000017,0.000018
17,-4.479124,-4.273960,0.000033,0.000053
18,-3.586030,-4.202767,0.000259,0.000063
19,-2.649558,-3.968340,0.002241,0.000108
21,-4.775985,-4.893146,0.000017,0.000013
22,-4.934794,-5.319127,0.000012,0.000005
26,-4.913284,-5.267832,0.000012,0.000005


In [26]:
result[result["y_true_10"]>0.0001]

,y_true,y_predicted,y_true_10,y_predicted_10
3,-3.889410,-4.013056,0.000129,0.000097
18,-3.586030,-4.202767,0.000259,0.000063
19,-2.649558,-3.968340,0.002241,0.000108
48,-3.340654,-3.856011,0.000456,0.000139
51,-3.716021,-3.651092,0.000192,0.000223
57,-3.913996,-4.506653,0.000122,0.000031
66,-3.346691,-3.882606,0.000450,0.000131
67,-3.501965,-3.741814,0.000315,0.000181
88,-3.266722,-4.287007,0.000541,0.000052
91,-3.511731,-3.863532,0.000308,0.000137
